#### <h1><center>CMSC 471: Introduction to Artificial Intelligence</center></h1>

<center><img src="img/title.jpeg" align="center"/></center>


<h3 style="color:blue;"><center>Instructor: Fereydoon Vafaei</center></h3>


<h5 style="color:purple;"><center>Adversarial Search and Games</center></h5>

<center><img src="img/UMBC_logo.png" align="center"/></center>

<h1><center>Adversarial Search and Games</center></h1>

<h5><center>"In which we explore environments where other agents are plotting against us."</center></h5>

- <ins>**Agenda**</ins>
    - Game Theory and Formal Definition of Game
    - The Minimax Search Algorithm
    - Alpha-Beta Pruning
    - Heuristic Alpha-Beta Tree Search
        - Evaluation Functions
    - Monte Carlo Tree Search MCTS
    - Stochastic Games
    - Limitations of Game Search Algorithms

<h1><center>Game Theory</center></h1>

- There are at least three stances we can take towards multi-agent environments.

- The first stance, appropriate when there are a very large number of agents, is to consider them in the aggregate as an economy, allowing us to do things like predict that increasing demand will cause prices to rise, without having to predict the action of any individual agent.

- Second, we could consider adversarial agents as just a part of the environment---a part that makes the environment nondeterministic. But if we model the adversaries in the same way that, say, rain sometimes falls and sometimes doesn't, we miss the idea that our adversaries are actively trying to defeat us, whereas the rain supposedly has no such intention.

- The third stance is to explicitly model the adversarial agents with the techniques of **adversarial game-tree search**. That is what we cover in our discussion of multi-agent adversarial environments.

<h1><center>Two-Player, Zero-Sum, Perfect Information Games</center></h1>

- The games most commonly studied within AI (such as chess and Go) are what game theorists call deterministic, two-player, turn-taking, **perfect information, zero-sum games**.

- "**Perfect information**" is a synonym for "fully observable", and "**zero-sum**" means that what is good for one player is just as bad for the other: there is no "win-win" outcome. We assume that the payoff to each player at the end of a game is equal and opposite.

- For games we often use the term **move** as a synonym for "action" and **position** as a synonym for "state".

- We call our two players `MAX` and `MIN`. `MAX` moves first, and then the players take turns moving until the game is over.

- At the end of the game, points are awraded to the winning player and penalties are given to the loser.

<h1><center>Definition of Game</center></h1>

  * $S_0$: The **initial state**, which specifies how the game is set up at the start. 
  * TO-MOVE$(s)$: The player whose turn it is to move in state $s$.
  * ACTIONS$(s)$: The set of legal moves in state $s$.
  * RESULT$(s,a)$: The **transition model** which defines the state resulting from taking action $a$ in state $s$.
  * IS-TERMINAL$(s)$: A **terminal test**, which is true when the game is over and false otherwise. States where the game has ended are called **terminal states**.
  * UTILITY$(s,p)$: A **utility function** (aka objective function or payoff function)---payoff for player $p$ upon reaching state $s$. It defines the final numeric value to player $p$ when the game ends in terminal state $s$.
  
- The two players in a two-player game are called `MAX` and `MIN`. These names reflect the meaning of the UTILITY$(s,p)$  function, which is to be maximized by Player `MAX` and minimized by Player `MIN`. 

<h1><center>Game Tree</center></h1>

- We define the complete **game tree** as a search tree that follows every sequence of moves all the way to terminal state. 

- Figure 5.1 shows part of the **game tree** for tic-tac-toe. From the initial state, MAX has nine possible moves.

- Play alternates between MAX's placing an X and MIN's placing an O until we reach leaf nodes corresponding to terminal states such that one player has three squares in a row (win/lose) or all the squares are filled (draw).

<center><img src="img/fig-5-1.png" align="center"/></center>

<font size=1>From Russel & Norvig Textbook</font>

<h1><center>Optimal Decisions in Games</center></h1>

- MAX wants to find a sequence of actions leading to a win, but MIN has something to say about it. This means that MAX's strategy must be a conditional plan---a contingent strategy specifying a response to each of MAX's possible moves.

- For games with multiple outcome scores, we need an algorithm called **minimax search**.

- Given a game tree, the optimal strategy can be determined by working out the **minimax value** of each state in the tree, which we write as MINIMAX(s).

- The minimax value is the utility (for MAX) of being in that state, *assuming that both players play optimally* from there to the end of the game.

- The minimax value of a terminal state is just its utility.

<h1><center>MiniMax Example</center></h1>

<center><img src="img/fig-5-2.png" align="center"/></center>

<font size=1>From Russel & Norvig Textbook</font>

<h1><center>MINIMAX(s)</center></h1>

- The calculation of the MINIMAX$(s)$ value of a state $s$ can be summarized as

$$
\text{MINIMAX}(s) = \begin{cases}
UTILITY(s, MAX) & \text{if }IS-TERMINAL(s)\\
\max_{a\in Actions(s)} \text{MINIMAX}(RESULT(s,a)) & \text{if
}TO-MOVE(s) \text{ = MAX}\\
\min_{a\in Actions(s)} \text{MINIMAX}(RESULT(s,a)) & \text{if
}TO-MOVE(s) \text{ = MIN}
\end{cases}
$$

- In a non-terminal state, MAX prefers to move to a state of maximum value when it's MAX's turn to move, and MIN prefers a state of minimum value (that is, minimum value for MAX and thus maximum value for MIN).

<h1><center>MiniMax Search Algorithm</center></h1>

- **Minimax search algorithm** can find the best move for MAX by trying all actions and choosing the one whose resulting state has the highest MINIMAX value.

- The minimax algorithm performs a complete **depth-first** search (exploration) of the search tree.

- The textbook shows in Figure 5.3 the *minimax-decision* algorithm as a depth-first search that altenates between calling `max-value` and `min-value` functions.

- The partial search tree in the next slide illustrates the reasoning behind the concept of alternate layers minimizing and maximizing the utility value to back up a value from terminal states to non-terminal states.

<center><img src="img/fig-5-3.png" align="center"/></center>

<font size=1>From Russel & Norvig Textbook</font>

In [1]:
from IPython.display import IFrame
IFrame("minimax.pdf", width=1000, height=600)

<h1><center>Pruning</center></h1>

- The number of game states is exponential in the depth of the tree. No algorithm can completely eliminate the exponent, but we can sometimes cut it in half, computing the correct minimax decision without examining every state by **pruning** large parts of the tree that make no difference to the outcome.

- **Pruning** makes the search more efficient by ignoring portions of the search tree that make no difference to the optimal move. 

- Some of the search tree can be ignored (**pruned**) if we know we cannot find a better move from the best one found so far.

<h1><center>Alpha-Beta Pruning</center></h1>

- For each node, keep track of three values, Minimax value (the same value returned by Minimax algorithm), as well as $\alpha$ and $\beta$

- **$\alpha$ = the value of the best (i.e. highest value) choice we have found so far at any choice point along the path for MAX.**

  * Think: $\alpha$ = **"at least"**
  * Any value less than this is no use because we already now how to achieve **at least** a value of $\alpha$
  * $\alpha = Max(current value, new value)$
  * Initially, $- \infty$
  * $\alpha$ is updated only at MAX nodes


- **$\beta$ = the value of the best (i.e. lowest value) choice we have found so far at any choice point along the path for MIN.**

  * Think: $\beta$ = **"at most"**
  * $\beta = Min(current value, new value)$
  * Initially, $+ \infty$
  * $\beta$ is updated only at MIN nodes
  
- The span between $\alpha$ and $\beta$ progressively gets smaller.

- **Pruning Condition**: Any unvisited children/subtree of the node for which $\beta <= \alpha$ can be pruned.

<h1><center>Alpha-Beta Pruning Examples</center></h1>

[Alpha-Beta Pruning Practice](abTreePractice-master/index.html) --- This link is NOT shared! Refer to the examples that are shared in the Course Materials.

In [2]:
from IPython.display import IFrame
IFrame("alpha-beta-example.pdf", width=1000, height=600)

<h1><center>Heuristic Alpha-Beta Tree Search</center></h1>

- To make use of our limited computation time, we can cut off the search early and apply a heuristic **evaluation function** to states, effectively treating nonterminal nodes as if they were terminal.

- In other words, we replace UTILITY function with EVAL, which estimates a state's utility.

- We also replace the terminal test by a **cutoff test**, which must return true for terminal states, but is otherwise free to decide when to cut off the search, based on the search depth and any property of the state that it chooses to consider.

- That gives us the formula H-MINIMAX($s, d$) for the heuristic minimax value of state $s$ at search depth $d$:

$$
\text{H-MINIMAX}(s, d) = \begin{cases}
EVAL(s, MAX) & \text{if }IS-CUTOFF(s,d)\\
\max_{a\in Actions(s)} \text{H-MINIMAX}(RESULT(s,a),d+1) & \text{if
}TO-MOVE(s) \text{ = MAX}\\
\min_{a\in Actions(s)} \text{H-MINIMAX}(RESULT(s,a),d+1) & \text{if
}TO-MOVE(s) \text{ = MIN}
\end{cases}
$$

<h1><center>Evaluation Functions</center></h1>

- A heuristic evaluation function EVAL(*s,p*) returns an **estimate** of the expected utility of state *s* to player *p*, just as the heuristic functions of Chapter 3 return an estimate of the distance to the goal.

- For terminal states, it must be that EVAL($s,p$)=UTILITY($s,p$) and for nonterminal states, the evaluation must be somewhere between a loss and a win:

    > UTILITY($loss,p$) $\leq$ EVAL($s,p$) $\leq$ UTILITY($win,p$)

- **Evaluation functions** are used to evaluate the "goodness" of a game position.

<h1><center>Evaluation Function for Chess</center></h1>

- Claude Shannon's paper *Programming a Computer for Playing Chess (1950)* was among the first proposals to apply evaluation functions to states in the search.


- Alan Turing’s function for chess:
> $f(n) = w(n) / b(n)$ where $w(n)$ is sum of point value of white’s pieces and $b(n)$ is black's pieces.


- Chess players have developed ways of judging the value of a position. Traditional piece values in introductory chess books are as follows: pawn:1; knight:3; bishop:3; rook:5; queen:9

<h1><center>Evaluation Function Formulation</center></h1>

- Mathematically, this kind of evaluation function is called **weighted linear function** because it can be expressed as:

$$Eval(s) = w_1f_1(s) + w_2f_2(s) + ... w_nf_n(s) = \sum_{i=1}^n w_i f_i$$

> where each $w_i$ is a weight and each $f_i$ is a feature of the position. For chess, $f_i$ could be the numbers of each kind of piece on the board and the $w_i$ could be the values of the pieces.

- IBM’s chess program [Deep Blue](https://en.wikipedia.org/wiki/Deep_Blue_(chess_computer)) (circa 1996) had $>8K$ features in its evaluation function.

- In DeepBlue’s alpha-beta pruning, average branching factor at node was ~6 instead of ~35! 

<h1><center>IBM Deep Blue</center></h1>

<center><img src="img/deepblue.jpg" align="center"/></center>

<h1><center>Evaluation Function Estimation</center></h1>

- In games where not so much experience is available like chess, the weights of the evaluation function can be estimated by the machine learning techniques.

<h1><center>Limitations of Alpha-Beta Pruning</center></h1>

- Despite the strength of Alpha-Beta pruning in search tree reduction, it has two major weaknesses:

    - First, if branching factor of tha game tree is too high (say 361 in Go game), alpha-beta would be limited to only 4 or 5 ply.

    - Second, it is difficult to define a good evaluation function for some games like Go.
    

- In response to these challenges, some modern game programs have abandoned alpha-beta search and instead use a strategy called **Monte Carlo Tree Search MCTS**.

<h1><center>Monte Carlo Tree Search MCTS</center></h1>

- The basic MCTS strategy does not use a heuristic function evaluation. Instead, the value of a state is estimated as the average utility over a number of **simulations** of complete games starting from the state.


- A **simulation** (also called a **playout** or **rollout**) chooses moves first for one player, then for the other, repeating until a terminal position is reached. At that point, the rules of the game (not fallible heuristics) determine who has won or lost and by what score.

<h1><center>Exploration-Exploitation Tradeoff in MCTS</center></h1>

- Exploration-Exploitation tradeoff in MCTS is done iteratively through four steps:
    - Selection
    - Expansion
    - Simulation
    - Backpropagation

<h1><center>UCT and UCB1</center></h1>

- One very effective **selection** policy is called **upper confidence bounds applied to trees** or **UCT**. The policy ranks each possible move based on an upper confidence bound formula called **UCB1**.

$$UCB1(n) = \frac{U(n)}{N(n)} + C \times \sqrt{\frac{log N(PARENT(n))}{N(n)}}$$

- where:
    - $U(n)$ is the total utility of all playouts that went through node $n$
    - $N(n)$ is the number of playouts through node $n$
    - $PARENT(n)$ is the parent node of node $n$
    - $\frac{U(n)}{N(n)}$ is the **exploitation term**: the average utility of $n$
    - $C \times \sqrt{\frac{log N(PARENT(n))}{N(n)}}$ is the **exploration term**: it has the count $N(n)$ in the denominator, which means the term will be high for the nodes that have only been explored a few times

<h1><center>MCTS Simulation Example</center></h1>

- MCTS maintains a search tree and grows it on each iteration of the following four steps:
- **Selection:** Starting at the root of the search tree, we choose a move (guided by the selection policy), leading to a successor node, and repeat that process, moving down the tree to a leaf. Figure 5.10(a) shows a search tree with the root representing a state where white has just moved, and white has won 37 out of the 100 playouts done so far. The thick arrow shows the selection of a move by black that leads to a node where black has won 60/79 playouts.
- **Expansion:** We grow the search tree by generating a new child of the selected node.
- **Simulation:** We perform a playout from the newly generated child node, choosing moves for both players according to the playout policy.
- **Back-propagation:** We now use the result of the simulation to update all the search tree nodes going up to the root.

<center><img src="img/fig-5-10.png" align="center"/></center>

<h1><center>Stochastic Games</center></h1>

- **Stochastic games** bring us closer to the unpredictability of real life by including a random element, such as the throwing of dice. 

- Backgommon is a typical stochastic game that combines luck and skill.

- A game tree in backgommon must include **chance nodes** in addition to MAX and MIN nodes.

<h1><center>Stochastic Games - Chance Nodes</center></h1>

- A stochastic game is modeled by simply adding a level of **chance nodes** between each player's levels in the search tree.

<center><img src="img/fig-5-13.png" align="center"/></center>

<font size=1>Image from Russel & Norvig Textbook</font>

<h1><center>Expected Value and Expectiminimax Value</center></h1>

- The next step is to understand how to make correct decisions.

- Obviously, we still want want to pick the move that leads to the best position. However, positions do not have definite minimax values.

- Instead, we can only calculate the **expected value** of a position: the average over all possible outcomes of the chance nodes.

- This leads us to the **expectiminimax value** for games with chance nodes, a generalization of the minimax value for deterministic games. 

- Terminal nodes and MAX and MIN nodes work exactly the same way as before with the caveat that the legal moves for MAX and MIN will depend on the outcome of the dice roll in the previous chance node.

- For chance nodes we compute the **expected value**, which is the sum of of the value over all outcomes, weighted by the probability of each chance action.

<h1><center>Expected Value Example - Dice Rolling</center></h1>

- An example of **expected value**:  The average value of a lot (infinite number) of dice rolls with a fair dice is

$$
(1+2+3+4+5+6) / 6
$$

- The **expected value** is exactly this average, but is defined as the sum of the possible values times their probability of occurring.

$$
1(1/6) + 2(1/6) + 3(1/6) + 4(1/6) + 5(1/6) + 6(1/6)
$$

- If, for example, the 4, 5 and 6 sides are less likely than the other sides, then the expected value might be:

$$
1(1/4) + 2(1/4) + 3(1/4) + 4(1/12) + 5(1/12) + 6(1/12)
$$

<h1><center>Stochastic Games Example - Expectiminimax Values</center></h1>

- The various outcomes from the chance node have certain probabilities of occurring.  When backing up values through a chance node, the values are multiplied by their probability of occurring.


- This illustrates  the **expectiminimax** values, for backing up values through chance nodes (the middle level with oval-shaped nodes).

<center><img src="img/expectedvalues.png" align="center"/></center>

<font size=1>Image from Professor Chuck Anderson's Notebooks - CSU</font>

<h1><center>Limitations of Game Search Algorithms</center></h1>

- One limitation of Alpha-beta is its vulnerability to errors in the heuristic function.


- A second limitation for both Alpha-beta and MCTS is that they are designed to calculate the values of legal moves. But sometimes there is one move that is obviously the best (only one legal move), and in that case, there is no point wasting computational time to figure out the value of the move.


- A third limitation is that both alpha-beta and MCTS do all their reasoning at the level of individual moves. Clearly, humans play games differently: they can reason at a more abstract level, considering a higher-level goal---for example trapping the opponent's queen---and using the goal to selectively generate plausible plans.


- A fourth issue is the ability to incorporate **Machine Learning** into the game search process. Early game programs relied on human expertise to hand-craft evaluation functions. Nowadays, more games rely on machine learning from self-play rather than game-specific human-generated expertise.

<h1><center>Adversarial Search Summary</center></h1>

- Games Assumptions: Zero-Sum and Perfect Information

- Definition of Games

- Minimax Search Algorithm

- Alpha-Beta Pruning

- Evaluation Functions

- Stochastic Games

- Limitations of Game Search Algorithms

<h1><center>Credit</center></h1>

- Some texts of these slides are directly quoted from AIMA textbook 4th Edition by Russel and Norvig.